# Reinforcement Learning: MDPs and Policy
In this part of the project, we will be using OpenAI's gym API for various reinforcement learning exercises.

In this part of the project, we will explore each of the following topics:

1. Markov Decision Processes (2 Hours)
    * States
    * Actions
    * Discount Factors
    * Transition Function
    * Reward Function
    * Introduction to OpenAI Gym API
2. Solving Markov Decision Processes (1 Hour)
    * Simple MDP
    * Policies
    * Value Iteration
    * Policy Extraction
    * Policy Iteration
    * Policy Evaluation
3. Reinforcement Learning (2 Hours)
    * Online Planning vs Offline Planning
    * Online Planning: Model-Based vs Model-Free Approaches
    * Model-Free: A Closer Look
    * Model-Free Policy Evaluation: Direct Evaluation (Passive RL)
    * Model-Free Policy Evaluation: Temporal Difference Learning (Passive RL)
    * Model-Free Policy Learning/Updates: Q-Learning (Active RL)
4. Wrapping Up
    * Opportunities in RL
    
The majority of the time you will be spending on this project will be in writing the various agents to solve the various gym environments. Throughout this notebook, you will be asked to reflect on various demonstrations. 
    

## Part 1: Markov Decision Processes (2 hour)

# TODO: Finish MDP explanation


### What is a Markov Decision Process? (30 min)
We can represent the world (like Taxi World and like Chain World) with a model. This model is called a Markov Decision Process. It has a number of components. Let's take a look at each of these components for a model we call chain world.

1. What are States?

2. What are Actions?

3. What are Transition Functions?

4. What are Rewards?

5. What are Discount Factors?

6. What are Q-States, Q-Values?


### OpenAI Gym API (30 mins)
The OpenAI Gym API provides an easy way for students and researchers to test out their Reinforcement Learning agents in pre-specified environments. The API also provides a framework to build your own environments to train your agents on. Throughout this project, you will be using the OpenAI Gym API. Here's a quick tutorial: 

### Introduction to OpenAI Gym Through Chain World

To create chain world, we built our own environment using the OpenAI Gym API. An environment is its own simulated world where RL agents can explore, learn, and reap rewards. Every environment is defined just as we would define an MDP -- with states, action spaces, slippage probabilities (transition functions), rewards, etc.

Please take a few minutes to look over the various environments included in the API at this link: [OpenAI Environments](https://gym.openai.com/envs/#classic_control). 

**Please carefully follow the code in the cells below and read the comments to get a full picture of the API you will be using throughout this part of the project.**

In [8]:
# Importing gym
import gym

# Installing the chain world environment
!pip install -e ./gym-note4-mdp/ -q

# Importing chain world environment
import gym_note4_mdp

In [9]:
# Making the environment
env = gym.make('note4-mdp-v0')

In [10]:
# Let's take a look at this environment
env.render()

Environment
+---------+
|B: : : :S|
+---------+


Using the `render()` function, you can visualize the chain world environment. This visualization has three parts. The _Environment_ part shows the agent in the environment. The _States_ part is a reference to quickly cross check which state the agent is in. The _Agent_ part shows the agent.

**Let's go over the specifics of the chain world as we are learning the API. Look to the code blocks for specific functions.**

### Chain World

This is a simple MDP which represents a chain of states. 

#### States

The states are 0, 1, 2, 3, and 4 and they correspond to the position of the agent (the yellow rectangle) in the chain. Initially the agent starts deterministically at State 2. Verify that the setup makes sense by running the cell below.

In [11]:
# The env.reset() method reinitializes the env to default values
env.reset()
env.render()
# We can access the environments's current state by using env.state
print("Agent's current state:", env.state)

Environment
+---------+
|B: : : :S|
+---------+
Agent's current state: 2


#### Actions

In the chain world, there are three actions that the agent can take:
     
* (1) forward, which moves the agent forward along the chain
* (0) backward, which moves the agent backward along the chain
* (2) exit, which tries to exit the world

The cell below will show you how to make actions using the `step(action)` function and will take you through one episode where the agent goes forward, backward, backward, backward, and then exits with reward +10.

In [12]:
env.reset()
env.discount_factor = 1
done = False

env.slip = 0 # This is the probability of the agent slipping
print("Frame 1: Initial State")
env.render(done=done)

state, reward, done, _ = env.step(1) # .step(action) function takes an action on the environment
print("\nFrame 2: Agent moves {} to state {} and receives reward {}".format("forward", state, reward))
env.render(done=done)

state, reward, done, _ = env.step(0) # moves backward
print("\nFrame 3: Agent moves {} to state {} and receives reward {}".format("backward", state, reward))
env.render(done=done)

state, reward, done, _ = env.step(0) # moves backward
print("\nFrame 4: Agent moves {} to state {} and receives reward {}".format("backward", state, reward))
env.render(done=done)

state, reward, done, _ = env.step(0) # moves backward
print("\nFrame 5: Agent moves {} to state {} and receives reward {}".format("backward", state, reward))
env.render(done=done)

state, reward, done, _ = env.step(2) # exits
print("\nFrame 6: Agent moves {} to state {} and receives reward {}".format("exit", state, reward))
env.render(done=done)



Frame 1: Initial State
Environment
+---------+
|B: : : :S|
+---------+

Frame 2: Agent moves forward to state 3 and receives reward 0
Environment
+---------+
|B: : : :S|
+---------+

Frame 3: Agent moves backward to state 2 and receives reward 0
Environment
+---------+
|B: : : :S|
+---------+

Frame 4: Agent moves backward to state 1 and receives reward 0
Environment
+---------+
|B: : : :S|
+---------+

Frame 5: Agent moves backward to state 0 and receives reward 0
Environment
+---------+
|B: : : :S|
+---------+

Frame 6: Agent moves exit to state 0 and receives reward 10
Environment
+---------+
|B: : : :S|
+---------+


#### Transition Functions

From any state, the agent can try to move fowards (1), move backwards (0), or exit (2). But there is no guarantee that the agent will succeed. You can imagine that chain world is a slippery place. The transitions are not determistic. If you choose to go forwards, you could slip and go backwards with some probability and vice versa. Also, if the agent tries to exit from any state other than the 0 or 4, nothing will happen because that is not valid action.


In [16]:
env.reset()
env.slip = 0
print("Slippage probability in deterministic world: {}".format(env.slip))

env.reset() # By default the world is uncertain
print("Slippage probability in uncertain world: {}".format(env.slip))

Slippage probability in deterministic world: 0
Slippage probability in uncertain world: 0.2


#### Rewards
     
At the beginning of the chain (State 0) there is a big reward of +10 (B) that the agent will be rewarded if it chooses to exit (Action 2) when in State 0.

At the end of the chain (State 4) there is a small reward of +1 (S) that the agent will be rewarded if it chooses to exit (Action 2) when in State 4.


In [17]:
print("Small Reward:", env.small)
print("Big Reward:", env.large)

Small Reward: 1
Big Reward: 10


#### Discount Factor
    
The discount factor is $\gamma = 0.1$. So as we take steps the rewards are discounted such that $${reward_{t+1}} = {reward_t}*{\gamma}$$ 

Observe what happens to the small and large rewards as we take steps in the cells below for various discount factors. Describe what you see in the cell below.

In [18]:
env.reset()
for discount_factor in [0.1, 0.5, 0.9]:
    env.reset()
    env.discount_factor = discount_factor
    print("Discount Factor: {} \n".format(env.discount_factor))
    print("\t After {} moves".format(0))
    print("\t Small Reward: {}".format(env.small))
    print("\t Big Reward: {} \n".format(env.large))

    action_space = [0, 1, 2]
    for i in range(1, 5):
        action = env.sample() #This lets us randomly sample an action
        env.step(action)
        print("\t After {} moves".format(i))
        print("\t Small Reward: {}".format(env.small))
        print("\t Big Reward: {} \n".format(env.large))
    env.step(1)

Discount Factor: 0.1 

	 After 0 moves
	 Small Reward: 1
	 Big Reward: 10 

	 After 1 moves
	 Small Reward: 0.1
	 Big Reward: 1.0 

	 After 2 moves
	 Small Reward: 0.010000000000000002
	 Big Reward: 0.1 

	 After 3 moves
	 Small Reward: 0.0010000000000000002
	 Big Reward: 0.010000000000000002 

	 After 4 moves
	 Small Reward: 0.00010000000000000003
	 Big Reward: 0.0010000000000000002 

Discount Factor: 0.5 

	 After 0 moves
	 Small Reward: 1
	 Big Reward: 10 

	 After 1 moves
	 Small Reward: 0.5
	 Big Reward: 5.0 

	 After 2 moves
	 Small Reward: 0.25
	 Big Reward: 2.5 

	 After 3 moves
	 Small Reward: 0.125
	 Big Reward: 1.25 

	 After 4 moves
	 Small Reward: 0.0625
	 Big Reward: 0.625 

Discount Factor: 0.9 

	 After 0 moves
	 Small Reward: 1
	 Big Reward: 10 

	 After 1 moves
	 Small Reward: 0.9
	 Big Reward: 9.0 

	 After 2 moves
	 Small Reward: 0.81
	 Big Reward: 8.1 

	 After 3 moves
	 Small Reward: 0.7290000000000001
	 Big Reward: 7.29 

	 After 4 moves
	 Small Reward: 0.6561000

**Check your understanding #1:** What impact does changing the discount factor have? Why do we need a discount factor in the first place? (Hint: What happens when the discount factor = 1?)



Your answer here

**Check your understanding #2:** What would you do if you were the agent in chain world? Think about this question in a general way. For example, what would you do if you were initially in state 4 and the discount factor was 0.1? What would you do if the slippage probability was greater than 0.5? We don't expect rigorous justification here, we just want to get you thinking about _policy_ -- a topic we will be looking at in much greater detail later on. 

Your answer here

## Taxi World

Now that you've gotten a chance to familiarize yourself with the OpenAI Gym API and the simple chain world environment, let's take a look at a more complicated (albeit still very simple) environment we call Taxi World. Fill in the code when prompted. 

After looking at the Taxi World in depth, you will be writing a "dumb agent" that chooses moves randomly for Taxi World and Chain World that will serve as a baseline for better agents we write later on.

In [19]:
# Install packages
import numpy as np
!pip install cmake 'gym[atari]' scipy -q

In [20]:
# Make the taxi world environment
env = gym.make("Taxi-v3").env

# Render the environment using the OpenAI Gym API
## Your code here ## (Ans: env.render())
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In the Taxi World, the agent is a taxi cab. In the visualization, the taxi cab is the yellow rectangle. The Taxi World has walls, represented by | and the taxi agent cannot go through these walls. The Taxi World also has pickup/dropoff locations represented by the letters (R, G, Y, B) in the visualization. 

In each run of Taxi World, the taxi agent must pickup a passenger at the blue letter location, transport them to the pink letter location, and dropoff the passenger at the pink letter location.

When the Taxi has picked up a passenger, its color will change from yellow to blue.

Formally, we can represent this as an MDP! Let's specify each component of the underlying MDP.

* **States**
    * How many states are there in the Taxi World?
    * The Taxi World environment has many more states than the Chain World environment. For the chain world, it was relatively easy to see that the chain world had 6 states (0, 1, 2, 3, 4, terminal state), but how would we figure out the number of states in the Taxi World?
    * **Let's go through this exercise:**

    * How many possible locations are there for the taxi agent?
        * Answer: The taxi world is a 5x5 grid so there are 25 possible locations.
    * For each location of the taxi, how many possible passenger locations are there?
        * Answer: There are 4 possible passenger locations if the passenger has not yet been picked up and 1 possible passenger location if the passenger has been picked up. This gives us a total of 5 passenger locations.
     
    * For each location of the taxi and passenger location, how many drop off locations are there?
        * Answer: There are still 4 possible drop off locations
    * Answer: So, in total there are $5*5*5*4 = 500$ states in this world! That's a lot more than there were in the chain world!  

* **Actions**
    * 0: south
    * 1: north
    * 2: east
    * 3: west
    * 4: pickup
    * 5: dropoff

* **Transitions**
    * For now, the taxi world is deterministic, so the taxi will go to the next state based solely on the action chosen by the taxi agent.
    
* **Rewards**
    * Movement actions have reward -1.0. A negative reward makes moving costly. This makes sense, because the taxi agent should be incentivized to pickup and dropoff passengers as efficiently as possible.
    * Pickup/dropoff actions have reward -10 if the taxi agent is not at a pickup or dropoff location. This is a large penalty.
    * At dropoff/pickup locations, dropoff and pickup would have higher rewards (+20), if the agent succeeds in picking up and dropping off the passenger at the correct locations.
    
* **Discount Factor**
    * The discount factor is 1. The rewards do not diminish over time.





## Coding Exercise: Random Agents (30 mins)

Now you will be using what you have learned to write code for an agent that moves randomly through the world. The recap section below provides some code that should be helpful. In the cell below, write code for a taxi agent that at each step chooses a random move. While the taxi agent has not dropped off its passenger at the correct dropoff location, continue making random moves.

In [21]:
# Recap of API
print("Action space: {}".format(env.action_space)) # Gives you the action space
print("Size of action space: {}".format(env.action_space.n))
print("Initial State")
env.render()

action = env.action_space.sample()# Sample an action randomly from the sample space
print("\nMake a random action: {}".format(action))
state, reward, done, _ = env.step(action) # Take an action
print(f"\n State: {state}, Reward: {reward}, Done: {done}")
env.render() # Draw the state

# state is the new state
# reward is the reward associated with making that action
# done is a boolean telling us whether we have reached a terminal state

Action space: Discrete(6)
Size of action space: 6
Initial State
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


Make a random action: 0

 State: 248, Reward: -1, Done: False
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)


Motivate RL With Random moving agent in Chain World. Have students code the random moving behavior in both of these agents. (15 min)


In [22]:
## YOUR CODE FOR RANDOM TAXI WORLD AGENT HERE ##
def random_agent(env):
    frames = []
    env.reset()
    ## Keep track of actions_taken, and total_reward in your code
    actions_taken = 0
    total_reward = 0
    done = False
    
    
    # Answer #
    # TODO: change the condition
    while not done:
        action = env.action_space.sample()
        state, reward, done, _ = env.step(action)
        total_reward += reward
        actions_taken += 1
        
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward,
            'total reward': total_reward
            }
        )
          
    ## END OF YOUR CODE ##
        
    return frames, actions_taken, total_reward
    
## Utils Functions to Run and Visualize ## 
# Source: see reference [1]
from IPython.display import clear_output
from time import sleep

def print_frames(frames, t=0.1):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Time: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        print(f"Total Reward: {frame['total reward']}")
        sleep(t)

In [24]:
frames, actions_taken, total_reward = random_agent(env)
print("Actions taken: {}".format(actions_taken))
print("Total Reward: {}".format(total_reward))

Actions taken: 459
Total Reward: -1680


In [25]:
# Hit ctrl-c to interrupt this if you would rather not watch the agent the whole time
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Time: 459
State: 410
Action: 5
Reward: 20
Total Reward: -1680


What can you say about the random agent in the Taxi world? About how many actions does the taxi agent take before picking up the passenger for the first time? How many actions does the random agent take before finally stumbling into the terminal state? How much reward does it incurr in total? 


Your answer here

The terrible performance of this random agent should motivate our study of reinforcement learning. The random agents performance in our taxi world will serve as our baseline for future agents we write.

For good measure, to showcase that the algorithm would work on any environment, run the random agent in the chain world environment in the cell below.

In [26]:
# Reimplemented with different sample method
# Get the action space
def agent(env):
    env.reset()
    frames, actions_taken, total_reward, done = [], 0, 0, False
    
    while not done:
        action = env.sample()
        state, reward, done, _ = env.step(action)
        total_reward += reward
        actions_taken += 1

        frames.append({
            'frame': env.render(mode='ansi', done=done),
            'state': state,
            'action': action,
            'reward': reward,
            'total reward': total_reward
            }
        )
    return frames, actions_taken, total_reward

env = gym.make('note4-mdp-v0')
frames, actions_taken, total_reward = agent(env)
print("Actions taken: {}".format(actions_taken))
print("Total Reward: {}".format(total_reward))

Actions taken: 5
Total Reward: 1.0000000000000004e-05


In [27]:
print_frames(frames)

Environment
+---------+
|B: : : :S|
+---------+

Time: 5
State: 4
Action: 2
Reward: 1.0000000000000004e-05
Total Reward: 1.0000000000000004e-05


The moral of this exercise is that we can do better! We can do better by learning from our experience. Let's dive in!

#### Student Exercise: Define the MDP for Black Jack. (10 min)

#### Student Exercise II: Define the MDP for Pacman. (10 min)

## Part 2: Solving MDPs  (1 Hour)

### Solving MDPs

To solve an MDP is to find an optimal policy. What is a policy?

#### Student Exercise: Test out different policies in the MDP provided

Specify the gym-note4-env and have students visualize different policies, enter their own policies, and see the effect of the discount factor on changing the optimal policy. This should be very hand wavy because students do not really know about Policy updates, Bellman, Value Iteration, etc.

### Bellman Equation

In [ ]:
# TODO 
# Justify why the equation is what it is.
# Draw out each part of the equation

### Value Iteration

Explain value iteration.
Do value iteration on the note4 simple MDP.

In [ ]:
# TODO

### Policy Extraction

In [ ]:
# TODO

### Policy Iteration

In [ ]:
# TODO

### Policy Evaluation


In [4]:
# TODO (Will be getting into Direct evaluation and temporal 
# difference learning as a way of evaluation later)

## Part 3: Reinforcement Learning: Evaluating an Learning Policies! (2 Hours)

By now students should have experience writing random moving agents, working with OpenAI gym API, and solving MDPs using value iteration, and learning optimal policies by hand on simple chain world.

Let's apply what we have learned to the Chain World and Taxi World by doing policy extraction, policy iteration, and policy evaluation in chain world. And then learning in both chain world and taxi world.

### Offline vs Online Planning

What if we don't know the underlying MDP?

### Online Planning: Model-Based

In [ ]:
# TODO think of something for students to do here.

### Online Planning: Model-Free

In [7]:
#### Online Planning: How would we do policy evaluation?
# 1. Direct Evaluation
# 2. Temporal Difference Learning

### Take students through chain world direct evaluation.
### Have them define some policies and try them out, 
# change slippage factor, change discount factor. Visualize in tables.


### Take students through temporal difference learning. 
### Have them implement the update rule. 
### Have them add a way of tuning hyperparameters like alpha (learnnig rate).


In [8]:
#### But wait isn't the goal to learn a policy?
#### Enter Q-Learning!

### Q-Learning